In [1]:
import json
import pandas as pd
import numpy as np
import datetime
from haversine import haversine

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))

In [3]:
TA_to_Check=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/check_single_zip/Single_Zip_TA_To_Check.xlsx",dtype=str)
TA_to_Check_list=list(TA_to_Check['TA'].apply(lambda x: x.zfill(3)))

In [4]:
len(TA_to_Check_list)

99

In [5]:
TA_to_Check_list[0]

'025'

In [6]:
TA_ta_level=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_2018-10-25.xlsx",dtype=str,sheetname="summary_by_TA")
TA_ta_level=TA_ta_level[TA_ta_level['TA'].isin(TA_to_Check_list)]
TA_ta_level['store_list']=TA_ta_level['store_list'].apply(lambda x: eval(x))
TA_ta_level=TA_ta_level.reset_index()
del TA_ta_level['index']

In [7]:
store_list_to_check=[]
for i in range(len(TA_ta_level)):
    store_list_to_check=store_list_to_check+TA_ta_level['store_list'][i]

In [8]:
TA_store_level=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_2018-10-25.xlsx",dtype=str,sheetname="output_TA_by_store")
TA_store_level=TA_store_level[TA_store_level['store_number'].isin(store_list_to_check)]
TA_store_level=TA_store_level[['store_number','TA','city','state','zip_code','latitude','longitude']]
TA_store_level=TA_store_level.reset_index()
del TA_store_level['index']
TA_store_level.head(1)

,store_number,TA,city,state,zip_code,latitude,longitude
0,1004,234,Fort. Polk,LA,71459,31.0456878,-93.2046098


In [9]:
TA_store_level['zip_code'].apply(lambda x: len(x)).unique()

array([5])

In [10]:
TA_store_level['Store_LatLng_to_Store_Zip_Center']=np.nan
for i in range(len(TA_store_level)):
    store_lat=float(TA_store_level['latitude'][i])
    store_lng=float(TA_store_level['longitude'][i])
    store_zip=TA_store_level['zip_code'][i]
    zip_center_of_store=zip_centers[store_zip]
    TA_store_level['Store_LatLng_to_Store_Zip_Center'][i]=haversine((store_lat,store_lng),zip_center_of_store,miles=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [11]:
for i in range(3,11):
    TA_store_level['zip_'+str(i)+"_miles_latlng"]=np.nan
for i in range(3,11):
    TA_store_level['zip_'+str(i)+"_miles_storezip"]=np.nan

In [12]:
for i in range(len(TA_store_level)):
    store_lat=float(TA_store_level['latitude'][i])
    store_lng=float(TA_store_level['longitude'][i])
    store_zip=TA_store_level['zip_code'][i]
    zip_center_of_store=zip_centers[store_zip]
    for j in range(3,11):
        locals()['zip_'+str(j)+"_miles_latlng"]=[]
        locals()['zip_'+str(j)+"_miles_storezip"]=[]
                
    for zip_cd in zip_centers.keys():
        distance_store_lat_long=haversine(zip_centers[zip_cd],(store_lat,store_lng),miles=True)
        distance_store_zip=haversine(zip_centers[zip_cd],zip_centers[store_zip],miles=True)
        for j in range(3,11):
            if distance_store_lat_long<=j:
                locals()['zip_'+str(j)+"_miles_latlng"]=locals()['zip_'+str(j)+"_miles_latlng"]+[zip_cd]
            if distance_store_zip<=j:
                locals()['zip_'+str(j)+"_miles_storezip"]=locals()['zip_'+str(j)+"_miles_storezip"]+[zip_cd]
    for j in range(3,11):
        TA_store_level['zip_'+str(j)+"_miles_latlng"][i]=locals()['zip_'+str(j)+"_miles_latlng"]
        TA_store_level['zip_'+str(j)+"_miles_storezip"][i]=locals()['zip_'+str(j)+"_miles_storezip"]
    if i % 10 ==1:
        print(i)
        

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1
11
21
31
41
51
61
71
81
91
101


In [13]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/TA/check_single_zip/Single_Zip_Store_Checked_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
TA_store_level.to_excel(writer,index=False)
writer.save()